In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel('charc_firewood_adjusted_transition.xlsx')
VERs_all=pd.read_excel('VERS.xlsx')

In [2]:
df['protocol_type'] = df['protocol_type'].replace('CDM AMS II.G','CDM AMS II G')

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [4]:
unique_stoves = df['stove'].unique()
for stove in unique_stoves[1:]:
    var_name = f"{stove.lower()}_VERs"
    globals()[var_name] = df.loc[df['stove'] == stove, 'verified_credits_per_stove'].sum()
    globals()[f"{stove}_N"] = 0
    globals()[f"{stove}_VERs_agw"] = []
    globals()[f"{stove}_ratio_analysis_project_mean_compilation"] = []
    globals()[f"{stove}_ratio_project_analysis_mean_compilation"] = []
    globals()[f"{stove}_ratio_analysis_project_lower_compilation"] = []
    globals()[f"{stove}_ratio_analysis_project_upper_compilation"] = []
    globals()[f"{stove}_ratio_project_analysis_lower_compilation"] = []
    globals()[f"{stove}_ratio_project_analysis_upper_compilation"] = []

# create variables for each country
unique_countries = df['country'].unique()
for country in unique_countries[1:]:
    var_name = f"{country.lower()}_VERs"
    globals()[var_name] = df.loc[df['country'] == country, 'verified_credits_per_stove'].sum()
    # lower the country name
    var_base_name = country.lower().replace(" ", "").replace("-", "").replace("'", "").replace(" ", "")  # lower the country name
    globals()[f"{var_base_name}_VERs_agw"] = []  # 'country_VERs_agw' list
    globals()[f"{var_base_name}_N"] = 0 # 'country_N'
    globals()[f"{var_base_name}_ratio_analysis_project_mean_compilation"] = [] # 'country_ratio_analysis_project_mean_compilation' list
    globals()[f"{var_base_name}_ratio_project_analysis_mean_compilation"] = [] # 'country_ratio_project_analysis_mean_compilation' list
    globals()[f"{var_base_name}_ratio_analysis_project_lower_compilation"] = [] # 'country_ratio_analysis_project_lower_compilation' list
    globals()[f"{var_base_name}_ratio_analysis_project_upper_compilation"] = []   # 'country_ratio_analysis_project_upper_compilation' list
    globals()[f"{var_base_name}_ratio_project_analysis_lower_compilation"] = [] # 'country_ratio_project_analysis_lower_compilation' list
    globals()[f"{var_base_name}_ratio_project_analysis_upper_compilation"] = [] #  'country_ratio_project_analysis_upper_compilation' list

# create variables for each type
unique_types = df['type'].unique()

for type1 in unique_types[1:]:
    globals()[f"{type1}_N"] = 0
    globals()[f"{type1}_VERs_agw"] = []
    globals()[f"{type1}_ratio_analysis_project_mean_compilation"] = []
    globals()[f"{type1}_ratio_project_analysis_mean_compilation"] = []
    globals()[f"{type1}_ratio_analysis_project_lower_compilation"] = []
    globals()[f"{type1}_ratio_analysis_project_upper_compilation"] = []
    globals()[f"{type1}_ratio_project_analysis_lower_compilation"] = []
    globals()[f"{type1}_ratio_project_analysis_upper_compilation"] = []

# create lists for each protocol_type
unique_protocol_types = df['protocol_type'].unique()

for protocol_type in unique_protocol_types[1:]:
  var_protocol_type = protocol_type.replace(" ", "_").replace("_AMS", "")
  globals()[f"{var_protocol_type}_VERs_agw"] = list()
  globals()[f"{var_protocol_type}_ratio_analysis_project_mean_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_project_analysis_mean_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_analysis_project_lower_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_analysis_project_upper_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_project_analysis_lower_compilation"] = []
  globals()[f"{var_protocol_type}_ratio_project_analysis_upper_compilation"] = []

In [5]:
ALL_N = 0
all_VERS_agw = []
ratio_analysis_project_mean_compilation=[]
ratio_project_analysis_mean_compilation=[]
ratio_analysis_project_lower_compilation=[]
ratio_analysis_project_upper_compilation=[]
ratio_project_analysis_lower_compilation=[]
ratio_project_analysis_upper_compilation=[]

# For protocol wide over-estimation
GS_Meter_N=0
GS_N=0
CDM_II_G_N=0
CDM_I_E_N=0

In [6]:
#import relevant packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time

from scipy import stats as stats
from scipy.stats import triang

import seaborn as sns

#triangular distributions

np.random.seed(4)
N = 10000                # random numbers to generate



#Usage
#min_usage=0.16
#mean_usage=0.476
#max_usage=0.8
#n_usage=np.random.triangular(left=min_usage, mode=mean_usage, right=max_usage, size=(N))

# Renewable GWP from Whitman & Lehman

gwp_ch4_renewable=28 #tCO2e/tonne CH4 i.e., instead of typically 25

# Whitman & lehmann 2011 also cite a value for CO
# but we do not include CO in our analysis

gwp_ch4_agw=30
gwp_n2O_agw=298

eff_trad_firewood=0.14
eff_improv_firewood=.21
eff_trad_charcoal=0.2
eff_improv_charcoal=.22
eff_kerosene=.47
eff_LPG=.5
eff_ethanol=.47
eff_pellets=.5 #only two pellet projects is the mimi motto

ncv_firewood_agw=.016
ncv_charcoal_agw=.027
ncv_kerosene_agw=.043
ncv_LPG_agw=.045
ncv_ethanol_agw=.027
ncv_pellets_agw=.017

kg_per_liter_ethanol_agw=0.783 #kg/liter

ef_combustion_co2_firewood=95.1 #tCO2/TJ updated good
ef_combustion_ch4_firewood=0.58 #tCH4/TJ updated good
ef_combustion_n2O_firewood=0 #tN2O/TJ

ef_combustion_co2_charcoal=114.75 #tCO2/TJ updated good
ef_combustion_ch4_charcoal=0.255 #tCH4/TJ updated good
ef_combustion_n2O_charcoal=0 #tN2O/TJ good


ef_total_not_combustion_co2_charcoal=87.725 #tCO2/TJ updated good
ef_total_not_combustion_ch4_charcoal=2.06 #tCH4/TJ updated good
ef_total_not_combustion_n2O_charcoal=0.0075#tN2O/TJ good

ef_total_co2_kerosene=79.9 #tCO2/TJ good
ef_total_ch4_kerosene=.107 #tCH4/TJ good
ef_total_n2O_kerosene=0.000141 #tN2O/TJ good

ef_total_co2_LPG=82.8#tCO2/TJ good
ef_total_ch4_LPG=0.117 #tCH4/TJ good
ef_total_n2O_LPG=0.000188 #tN2O/TJ good

ef_total_co2_ethanol=60.45#tCO2/TJ good
ef_total_ch4_ethanol=0.61 #tCH4/TJ good
ef_total_n2O_ethanol=0.05 #tN2O/TJ good

ef_total_co2_pellets=90.05#tCO2/TJ good
ef_total_ch4_pellets=0.105#tCH4/TJ good
ef_total_n2O_pellets=0 #tN2O/TJ good

ef_total_zeroed_co2_ethanol=-9.78#tCO2/TJ good
ef_total_zeroed_ch4_ethanol=0.61 #tCH4/TJ good
ef_total_zeroed_n2O_ethanol=0.05 #tN2O/TJ good

ef_total_zeroed_co2_pellets=0 #tCO2/TJ good
ef_total_zeroed_ch4_pellets=0.105 #tCH4/TJ good
ef_total_zeroed_n2O_pellets=0 #tN2O/TJ good

In [7]:
i=1 # The row you want to evaluate
while i<len(df):
    # first row is units
    VERs=VERs_all.iloc[i,1]
    protocol_type=df['protocol_type'][i]

    protocol_id=df['protocol_id'][i]

    monitoring_period=df['monitoring_period'][i]

    verified_credits=df['verified_credits'][i]

    num_countries=df['num_countries'][i]

    prov_project=df['prov_project'][i]

    fNRB_project=df['fNRB_project'][i]

    mofus_fNRB_bailis=df['mofus_fNRB_bailis'][i]


    n_devices_domestic=df['n_devices_domestic'][i]

    n_devices_commercial=df['n_devices_commercial'][i]

    project_days=df['project_days'][i]

    project_days_commercial=df['project_days_commercial'][i]

    av_hhsize=df['av_hhsize'][i]

    verified_credits=df['verified_credits'][i]

    stove_year_cohort=df['stove_year_cohort'][i]

    # Baseline Fuel % What are the baseline fuels in this project location? [put 0% if no use in this location]
    prc_firewood_baseline=df['prc_firewood_baseline'][i]

    prc_charcoal_baseline=df['prc_charcoal_baseline'][i]

    prc_kerosene_baseline=df['prc_kerosene_baseline'][i]

    prc_lpg_baseline=df['prc_lpg_baseline'][i]

    prc_ethanol_baseline=df['prc_ethanol_baseline'][i]

    prc_electric_baseline=df['prc_electric_baseline'][i]

    prc_pellets_baseline=df['prc_pellets_baseline'][i]





    # Baseline Fuel % What are the baseline commercial fuels in this project location? [put 0% if no use in this location]
    prc_firewood_baseline_commercial=df['prc_firewood_baseline_commercial'][i]

    prc_charcoal_baseline_commercial=df['prc_charcoal_baseline_commercial'][i]

    prc_kerosene_baseline_commercial=df['prc_kerosene_baseline_commercial'][i]

    prc_lpg_baseline_commercial=df['prc_lpg_baseline_commercial'][i]

    prc_ethanol_baseline_commercial=df['prc_ethanol_baseline_commercial'][i]

    prc_electric_baseline_commercial=df['prc_electric_baseline_commercial'][i]

    prc_pellets_baseline_commercial=df['prc_pellets_baseline_commercial'][i]






    # Baseline Consumption: What is the baseline fuel consumption of each baseline fuel in this project location [put 0 if no use in this location]
    kg_firewood_baseline=df['kg_firewood_baseline'][i]

    kg_charcoal_baseline=df['kg_charcoal_baseline'][i]

    kg_kerosene_baseline=df['kg_kerosene_baseline'][i]

    kg_lpg_baseline=df['kg_lpg_baseline'][i]

    l_ethanol_baseline=df['l_ethanol_baseline'][i]

    kg_per_liter_ethanol_baseline=df['kg_per_liter_ethanol_baseline'][i]

    kWh_electric_baseline=df['kWh_electric_baseline'][i]

    kg_pellets_baseline=df['kg_pellets_baseline'][i]



    kg_firewood_baseline_commercial=df['kg_firewood_baseline_commercial'][i]

    kg_charcoal_baseline_commercial=df['kg_charcoal_baseline_commercial'][i]

    kg_kerosene_baseline_commercial=df['kg_kerosene_baseline_commercial'][i]

    kg_lpg_baseline_commercial=df['kg_lpg_baseline_commercial'][i]

    l_ethanol_baseline_commercial=df['l_ethanol_baseline_commercial'][i]

    kWh_electric_baseline_commercial=df['kWh_electric_baseline_commercial'][i]

    kg_pellets_baseline_commercial=df['kg_pellets_baseline_commercial'][i]


    # Stove Efficiencies at Baseline


    eff_firewood_baseline=df['eff_firewood_baseline'][i]

    eff_charcoal_baseline=df['eff_charcoal_baseline'][i]

    eff_kerosene_baseline=df['eff_kerosene_baseline'][i]

    eff_LPG_baseline=df['eff_LPG_baseline'][i]

    eff_ethanol_baseline=df['eff_ethanol_baseline'][i]

    eff_electric_baseline=df['eff_electric_baseline'][i]

    eff_pellets_baseline=df['eff_pellets_baseline'][i]


    # Commercial Baseline Stove Efficiences

    eff_firewood_baseline_commercial=df['eff_firewood_baseline_commercial'][i]

    eff_charcoal_baseline_commercial=df['eff_charcoal_baseline_commercial'][i]

    eff_kerosene_baseline_commercial=df['eff_kerosene_baseline_commercial'][i]

    eff_LPG_baseline_commercial=df['eff_LPG_baseline_commercial'][i]

    eff_ethanol_baseline_commercial=df['eff_ethanol_baseline_commercial'][i]

    eff_electric_baseline_commercial=df['eff_electric_baseline_commercial'][i]

    eff_pellet_baseline_commercial=df['eff_pellet_baseline_commercial'][i]


    # Caloric Values: What are the assumed calorific values  of the baseline fuels? (TJ/ton) [put 0 if not in use in this location]

    ncv_firewood_baseline=df['ncv_firewood_baseline'][i]

    ncv_charcoal_baseline=df['ncv_charcoal_baseline'][i]

    ncv_kerosene_baseline=df['ncv_kerosene_baseline'][i]

    ncv_LPG_baseline=df['ncv_LPG_baseline'][i]

    ncv_ethanol_baseline=df['ncv_ethanol_baseline'][i]

    ncv_pellets_baseline=df['ncv_pellets_baseline'][i]





    # GWP: What are the assumed global warming potentials of CH4 & N20? If not included put 0 or leave blank
    gwp_ch4= 25

    gwp_n2O=298


    # CO2 Emission Factors: What is the assumed carbon dioxide emission factor of the baseline fuels?
    #[if CDM write in the assumed EF of the projected fossil fuel]
    #[ put 0 if you do not account for CO2 EF at Baseline or this fuel is not applicable in your location]
    co2_ef_firewood=df['co2_ef_firewood'][i]
    co2_ef_charcoal=df['co2_ef_charcoal'][i]

    co2_ef_kerosene=df['co2_ef_kerosene'][i]

    co2_ef_LPG=df['co2_ef_LPG'][i]

    co2_ef_ethanol=df['co2_ef_ethanol'][i]

    co2_ef_electric=df['co2_ef_electric'][i]

    co2_ef_pellets=df['co2_ef_pellets'][i]


    #Non-CO2 Baseline Emission Factors

    nonco2_ef_firewood=df['nonco2_ef_firewood'][i]
    nonco2_ef_charcoal=df['nonco2_ef_charcoal'][i]

    nonco2_ef_kerosene=df['nonco2_ef_kerosene'][i]

    nonco2_ef_LPG=df['nonco2_ef_LPG'][i]

    nonco2_ef_ethanol=df['nonco2_ef_ethanol'][i]

    nonco2_ef_electric=df['nonco2_ef_electric'][i]

    nonco2_ef_pellets=df['nonco2_ef_pellets'][i]


    # Emissions from Stove Production

    stoveproduction_ef_co2_firewood_baseline= df['stoveproduction_ef_co2_firewood_baseline'][i]
    stoveproduction_ef_ch4_firewood_baseline= df['stoveproduction_ef_ch4_firewood_baseline'][i]
    stoveproduction_ef_n2O_firewood_baseline= df['stoveproduction_ef_n2O_firewood_baseline'][i]


    stoveproduction_ef_co2_charcoal_baseline=df['stoveproduction_ef_co2_charcoal_baseline'][i]
    stoveproduction_ef_ch4_charcoal_baseline=df['stoveproduction_ef_ch4_charcoal_baseline'][i]
    stoveproduction_ef_n2O_charcoal_baseline=df['stoveproduction_ef_n2O_charcoal_baseline'][i]


    stoveproduction_ef_co2_kerosene_baseline=df['stoveproduction_ef_co2_kerosene_baseline'][i]
    stoveproduction_ef_ch4_kerosene_baseline=df['stoveproduction_ef_ch4_kerosene_baseline'][i]
    stoveproduction_ef_n2O_kerosene_baseline=df['stoveproduction_ef_n2O_kerosene_baseline'][i]


    stoveproduction_ef_co2_LPG_baseline=df['stoveproduction_ef_co2_LPG_baseline'][i]
    stoveproduction_ef_ch4_LPG_baseline=df['stoveproduction_ef_ch4_LPG_baseline'][i]
    stoveproduction_ef_n2O_LPG_baseline=df['stoveproduction_ef_n2O_LPG_baseline'][i]


    stoveproduction_ef_co2_ethanol_baseline=df['stoveproduction_ef_co2_ethanol_baseline'][i]
    stoveproduction_ef_ch4_ethanol_baseline=df['stoveproduction_ef_ch4_ethanol_baseline'][i]
    stoveproduction_ef_n2O_ethanol_baseline=df['stoveproduction_ef_n2O_ethanol_baseline'][i]


    stoveproduction_ef_co2_electric_baseline=df['stoveproduction_ef_co2_electric_baseline'][i]
    stoveproduction_ef_ch4_electric_baseline=df['stoveproduction_ef_ch4_electric_baseline'][i]
    stoveproduction_ef_n2O_electric_baseline=df['stoveproduction_ef_n2O_electric_baseline'][i]


    stoveproduction_ef_co2_pellets_baseline=df['stoveproduction_ef_co2_pellets_baseline'][i]
    stoveproduction_ef_ch4_pellets_baseline=df['stoveproduction_ef_ch4_pellets_baseline'][i]
    stoveproduction_ef_n2O_pellets_baseline=df['stoveproduction_ef_n2O_pellets_baseline'][i]


    feedstockproduction_ef_co2_firewood_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_firewood_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_firewood_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    feedstockproduction_ef_co2_charcoal_baseline=df['feedstockproduction_ef_co2_charcoal_baseline'][i]
    feedstockproduction_ef_ch4_charcoal_baseline=df['stoveproduction_ef_ch4_charcoal_baseline'][i]
    feedstockproduction_ef_n2O_charcoal_baseline=df['feedstockproduction_ef_n2O_charcoal_baseline'][i]


    feedstockproduction_ef_co2_kerosene_baseline=df['feedstockproduction_ef_co2_kerosene_baseline'][i]
    feedstockproduction_ef_ch4_kerosene_baseline=df['feedstockproduction_ef_ch4_kerosene_baseline'][i]
    feedstockproduction_ef_n2O_kerosene_baseline=df['feedstockproduction_ef_n2O_kerosene_baseline'][i]


    feedstockproduction_ef_co2_LPG_baseline=df['feedstockproduction_ef_co2_LPG_baseline'][i]
    feedstockproduction_ef_ch4_LPG_baseline=df['feedstockproduction_ef_ch4_LPG_baseline'][i]
    feedstockproduction_ef_n2O_LPG_baseline=df['feedstockproduction_ef_n2O_LPG_baseline'][i]


    feedstockproduction_ef_co2_ethanol_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_ethanol_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_ethanol_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    feedstockproduction_ef_co2_electric_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_electric_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_electric_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    feedstockproduction_ef_co2_pellets_baseline=df['feedstockproduction_ef_co2_firewood_baseline'][i]
    feedstockproduction_ef_ch4_pellets_baseline=df['feedstockproduction_ef_ch4_firewood_baseline'][i]
    feedstockproduction_ef_n2O_pellets_baseline=df['feedstockproduction_ef_n2O_firewood_baseline'][i]


    distribution_ef_co2_firewood_baseline=df['distribution_ef_co2_firewood_baseline'][i]
    distribution_ef_ch4_firewood_baseline=df['distribution_ef_ch4_firewood_baseline'][i]
    distribution_ef_n2O_firewood_baseline=df['distribution_ef_n2O_firewood_baseline'][i]


    distribution_ef_co2_charcoal_baseline=df['distribution_ef_co2_charcoal_baseline'][i]
    distribution_ef_ch4_charcoal_baseline=df['distribution_ef_ch4_charcoal_baseline'][i]
    distribution_ef_n2O_charcoal_baseline=df['distribution_ef_n2O_charcoal_baseline'][i]


    distribution_ef_co2_kerosene_baseline=df['distribution_ef_co2_kerosene_baseline'][i]
    distribution_ef_ch4_kerosene_baseline=df['distribution_ef_ch4_kerosene_baseline'][i]
    distribution_ef_n2O_kerosene_baseline=df['distribution_ef_n2O_kerosene_baseline'][i]


    distribution_ef_co2_LPG_baseline=df['distribution_ef_co2_LPG_baseline'][i]
    distribution_ef_ch4_LPG_baseline=df['distribution_ef_ch4_LPG_baseline'][i]
    distribution_ef_n2O_LPG_baseline=df['distribution_ef_n2O_LPG_baseline'][i]


    distribution_ef_co2_ethanol_baseline=df['distribution_ef_co2_ethanol_baseline'][i]
    distribution_ef_ch4_ethanol_baseline=df['distribution_ef_ch4_ethanol_baseline'][i]
    distribution_ef_n2O_ethanol_baseline=df['distribution_ef_n2O_ethanol_baseline'][i]


    distribution_ef_co2_electric_baseline=df['distribution_ef_co2_electric_baseline'][i]
    distribution_ef_ch4_electric_baseline=df['distribution_ef_ch4_electric_baseline'][i]
    distribution_ef_n2O_electric_baseline=df['distribution_ef_n2O_electric_baseline'][i]


    distribution_ef_co2_pellets_baseline=df['distribution_ef_co2_pellets_baseline'][i]
    distribution_ef_ch4_pellets_baseline=df['distribution_ef_ch4_pellets_baseline'][i]
    distribution_ef_n2O_pellets_baseline=df['distribution_ef_n2O_pellets_baseline'][i]


    processing_ef_co2_firewood_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_ch4_firewood_baseline=df['processing_ef_ch4_firewood_baseline'][i]
    processing_ef_n2O_firewood_baseline=df['processing_ef_n2O_firewood_baseline'][i]


    processing_ef_co2_charcoal_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_ch4_charcoal_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_n2O_charcoal_baseline=df['processing_ef_co2_firewood_baseline'][i]


    processing_ef_co2_kerosene_baseline=df['processing_ef_co2_kerosene_baseline'][i]
    processing_ef_ch4_kerosene_baseline=df['processing_ef_ch4_kerosene_baseline'][i]
    processing_ef_n2O_kerosene_baseline=df['processing_ef_n2O_kerosene_baseline'][i]


    processing_ef_co2_LPG_baseline=df['processing_ef_co2_LPG_baseline'][i]
    processing_ef_ch4_LPG_baseline=df['processing_ef_ch4_LPG_baseline'][i]
    processing_ef_n2O_LPG_baseline=df['processing_ef_n2O_LPG_baseline'][i]


    processing_ef_co2_ethanol_baseline=df['processing_ef_co2_ethanol_baseline'][i]
    processing_ef_ch4_ethanol_baseline=df['processing_ef_ch4_ethanol_baseline'][i]
    processing_ef_n2O_ethanol_baseline=df['processing_ef_n2O_ethanol_baseline'][i]


    processing_ef_co2_electric_baseline=df['processing_ef_co2_electric_baseline'][i]
    processing_ef_ch4_electric_baseline=df['processing_ef_ch4_electric_baseline'][i]
    processing_ef_n2O_electric_baseline=df['processing_ef_n2O_electric_baseline'][i]


    processing_ef_co2_pellets_baseline=df['processing_ef_co2_firewood_baseline'][i]
    processing_ef_ch4_pellets_baseline=df['processing_ef_ch4_firewood_baseline'][i]
    processing_ef_n2O_pellets_baseline=df['processing_ef_n2O_firewood_baseline'][i]

    # Project Fuel % What are the project domestic fuels in this project location? [put 0% if no use in this location]
    prc_firewood_project=df['prc_firewood_project'][i]

    prc_charcoal_project=df['prc_charcoal_project'][i]

    prc_kerosene_project=df['prc_kerosene_project'][i]

    prc_lpg_project=df['prc_lpg_project'][i]

    prc_ethanol_project=df['prc_ethanol_project'][i]

    prc_electric_project=df['prc_electric_project'][i]

    prc_pellets_project=df['prc_pellets_project'][i]



    # Project Fuel % What are the project commercial fuels in this project location? [put 0% if no use in this location]
    prc_firewood_project_commercial=df['prc_firewood_project_commercial'][i]

    prc_charcoal_project_commercial=df['prc_charcoal_project_commercial'][i]

    prc_kerosene_project_commercial=df['prc_kerosene_project_commercial'][i]

    prc_lpg_project_commercial=df['prc_lpg_project_commercial'][i]

    prc_ethanol_project_commercial=df['prc_ethanol_project_commercial'][i]

    prc_electric_project_commercial=df['prc_electric_project_commercial'][i]

    prc_pellets_project_commercial=df['prc_pellets_project_commercial'][i]



    # Project Fuel Consumption

    kg_firewood_project=df['kg_firewood_project'][i]

    kg_charcoal_project=df['kg_charcoal_project'][i]

    kg_kerosene_project=df['kg_kerosene_project'][i]

    kg_lpg_project=df['kg_lpg_project'][i]

    l_ethanol_project=df['l_ethanol_project'][i]


    kg_per_liter_ethanol_project=df['kg_per_liter_ethanol_project'][i]

    kWh_electric_project=df['kWh_electric_project'][i]

    kg_pellets_project=df['kg_pellets_project'][i]


    # Project Insitutional Consumption


    kg_firewood_project_commercial=df['kg_firewood_project_commercial'][i]

    kg_charcoal_project_commercial=df['kg_charcoal_project_commercial'][i]

    kg_kerosene_project_commercial=df['kg_kerosene_project_commercial'][i]

    kg_lpg_project_commercial=df['kg_lpg_project_commercial'][i]

    l_ethanol_project_commercial=df['l_ethanol_project_commercial'][i]

    kWh_electric_project_commercial=df['kWh_electric_project_commercial'][i]

    kg_pellets_project_commercial=df['kg_pellets_project_commercial'][i]




    # Project Stove Efficiencies

    eff_firewood_project=df['eff_firewood_project'][i]

    eff_charcoal_project=df['eff_charcoal_project'][i]

    eff_kerosene_project=df['eff_kerosene_project'][i]

    eff_LPG_project=df['eff_LPG_project'][i]

    eff_ethanol_project=df['eff_ethanol_project'][i]

    eff_electric_project=df['eff_electric_project'][i]

    eff_pellet_project=df['eff_pellet_project'][i]


    # Project Instituitonal Stove Efficiencies

    eff_firewood_project_commercial=df['eff_firewood_project_commercial'][i]

    eff_charcoal_project_commercial=df['eff_charcoal_project_commercial'][i]

    eff_kerosene_project_commercial=df['eff_kerosene_project_commercial'][i]

    eff_LPG_project_commercial=df['eff_LPG_project_commercial'][i]

    eff_ethanol_project_commercial=df['eff_ethanol_project_commercial'][i]

    eff_electric_project_commercial=df['eff_electric_project_commercial'][i]

    eff_pellet_project_commercial=df['eff_pellet_project_commercial'][i]


    #Assumed Caloric Values in the Project Scenario

    ncv_firewood_project=df['ncv_firewood_project'][i]

    ncv_charcoal_project=df['ncv_charcoal_project'][i]

    ncv_kerosene_project=df['ncv_kerosene_project'][i]

    ncv_LPG_project=df['ncv_LPG_project'][i]

    ncv_ethanol_project=df['ncv_ethanol_project'][i]

    ncv_pellets_project=df['ncv_pellets_project'][i]





    # Assumed GWP potentials of non-Co2 gases in the project
    gwp_ch4_project=25

    gwp_n2O_project=298


    # CO2 Emission Factors in the Project

    co2_ef_firewood_project=df['co2_ef_firewood_project'][i]

    co2_ef_charcoal_project=df['co2_ef_charcoal_project'][i]

    co2_ef_kerosene_project=df['co2_ef_kerosene_project'][i]

    co2_ef_LPG_project=df['co2_ef_LPG_project'][i]

    co2_ef_ethanol_project=df['co2_ef_ethanol_project'][i]

    co2_ef_electric_project=df['co2_ef_electric_project'][i]

    co2_ef_pellets_project=df['co2_ef_pellets_project'][i]


    # Non-CO2 Emission Factors in the Project

    nonco2_ef_firewood_project=df['nonco2_ef_firewood_project'][i]

    nonco2_ef_charcoal_project=df['nonco2_ef_charcoal_project'][i]

    nonco2_ef_kerosene_project=df['nonco2_ef_kerosene_project'][i]

    nonco2_ef_LPG_project=df['nonco2_ef_LPG_project'][i]

    nonco2_ef_ethanol_project=df['nonco2_ef_ethanol_project'][i]

    nonco2_ef_electric_project=df['nonco2_ef_electric_project'][i]

    nonco2_ef_pellets_project=df['nonco2_ef_pellets_project'][i]

    # Stove Production Emission Factors


    stoveproduction_ef_co2_firewood_project=df['stoveproduction_ef_co2_firewood_project'][i]
    stoveproduction_ef_ch4_firewood_project=df['stoveproduction_ef_ch4_firewood_project'][i]
    stoveproduction_ef_n2O_firewood_project=df['stoveproduction_ef_n2O_firewood_project'][i]


    stoveproduction_ef_co2_charcoal_project=df['stoveproduction_ef_co2_charcoal_project'][i]
    stoveproduction_ef_ch4_charcoal_project=df['stoveproduction_ef_ch4_charcoal_project'][i]
    stoveproduction_ef_n2O_charcoal_project=df['stoveproduction_ef_n2O_charcoal_project'][i]


    stoveproduction_ef_co2_kerosene_project=df['stoveproduction_ef_co2_kerosene_project'][i]
    stoveproduction_ef_ch4_kerosene_project=df['stoveproduction_ef_ch4_kerosene_project'][i]
    stoveproduction_ef_n2O_kerosene_project=df['stoveproduction_ef_n2O_kerosene_project'][i]


    stoveproduction_ef_co2_LPG_project=df['stoveproduction_ef_co2_LPG_project'][i]
    stoveproduction_ef_ch4_LPG_project=df['stoveproduction_ef_ch4_LPG_project'][i]
    stoveproduction_ef_n2O_LPG_project=df['stoveproduction_ef_n2O_LPG_project'][i]


    stoveproduction_ef_co2_ethanol_project=df['stoveproduction_ef_co2_ethanol_project'][i]
    stoveproduction_ef_ch4_ethanol_project=df['stoveproduction_ef_ch4_ethanol_project'][i]
    stoveproduction_ef_n2O_ethanol_project=df['stoveproduction_ef_n2O_ethanol_project'][i]


    stoveproduction_ef_co2_electric_project=df['stoveproduction_ef_co2_electric_project'][i]
    stoveproduction_ef_ch4_electric_project=df['stoveproduction_ef_ch4_electric_project'][i]
    stoveproduction_ef_n2O_electric_project=df['stoveproduction_ef_n2O_electric_project'][i]


    stoveproduction_ef_co2_pellets_project=df['stoveproduction_ef_co2_pellets_project'][i]
    stoveproduction_ef_ch4_pellets_project=df['stoveproduction_ef_ch4_pellets_project'][i]
    stoveproduction_ef_n2O_pellets_project=df['stoveproduction_ef_n2O_pellets_project'][i]

    # Project Feedstock Emission Factors

    feedstockproduction_ef_co2_firewood_project=df['feedstockproduction_ef_co2_firewood_project'][i]
    feedstockproduction_ef_ch4_firewood_project=df['feedstockproduction_ef_ch4_firewood_project'][i]
    feedstockproduction_ef_n2O_firewood_project=df['feedstockproduction_ef_n2O_firewood_project'][i]


    feedstockproduction_ef_co2_charcoal_project=df['feedstockproduction_ef_co2_charcoal_project'][i]
    feedstockproduction_ef_ch4_charcoal_project=df['feedstockproduction_ef_ch4_charcoal_project'][i]
    feedstockproduction_ef_n2O_charcoal_project=df['feedstockproduction_ef_n2O_charcoal_project'][i]


    feedstockproduction_ef_co2_kerosene_project=df['feedstockproduction_ef_co2_kerosene_project'][i]
    feedstockproduction_ef_ch4_kerosene_project=df['feedstockproduction_ef_ch4_kerosene_project'][i]
    feedstockproduction_ef_n2O_kerosene_project=df['feedstockproduction_ef_n2O_kerosene_project'][i]


    feedstockproduction_ef_co2_LPG_project=df['feedstockproduction_ef_co2_LPG_project'][i]
    feedstockproduction_ef_ch4_LPG_project=df['feedstockproduction_ef_ch4_LPG_project'][i]
    feedstockproduction_ef_n2O_LPG_project=df['feedstockproduction_ef_n2O_LPG_project'][i]


    feedstockproduction_ef_co2_ethanol_project=df['feedstockproduction_ef_co2_ethanol_project'][i]
    feedstockproduction_ef_ch4_ethanol_project=df['feedstockproduction_ef_ch4_ethanol_project'][i]
    feedstockproduction_ef_n2O_ethanol_project=df['feedstockproduction_ef_n2O_ethanol_project'][i]


    feedstockproduction_ef_co2_electric_project=df['feedstockproduction_ef_co2_electric_project'][i]
    feedstockproduction_ef_ch4_electric_project=df['feedstockproduction_ef_ch4_electric_project'][i]
    feedstockproduction_ef_n2O_electric_project=df['feedstockproduction_ef_n2O_electric_project'][i]


    feedstockproduction_ef_co2_pellets_project=df['feedstockproduction_ef_co2_pellets_project'][i]
    feedstockproduction_ef_ch4_pellets_project=df['feedstockproduction_ef_ch4_pellets_project'][i]
    feedstockproduction_ef_n2O_pellets_project=df['feedstockproduction_ef_n2O_pellets_project'][i]


    # Project Distribution Emission Factors
    distribution_ef_co2_firewood_project=df['distribution_ef_co2_firewood_project'][i]
    distribution_ef_ch4_firewood_project=df['distribution_ef_ch4_firewood_project'][i]
    distribution_ef_n2O_firewood_project=df['distribution_ef_n2O_firewood_project'][i]


    distribution_ef_co2_charcoal_project=df['distribution_ef_co2_charcoal_project'][i]
    distribution_ef_ch4_charcoal_project=df['distribution_ef_ch4_charcoal_project'][i]
    distribution_ef_n2O_charcoal_project=df['distribution_ef_n2O_charcoal_project'][i]


    distribution_ef_co2_kerosene_project=df['distribution_ef_co2_kerosene_project'][i]
    distribution_ef_ch4_kerosene_project=df['distribution_ef_ch4_kerosene_project'][i]
    distribution_ef_n2O_kerosene_project=df['distribution_ef_n2O_kerosene_project'][i]


    distribution_ef_co2_LPG_project=df['distribution_ef_co2_LPG_project'][i]
    distribution_ef_ch4_LPG_project=df['distribution_ef_ch4_LPG_project'][i]
    distribution_ef_n2O_LPG_project=df['distribution_ef_n2O_LPG_project'][i]


    distribution_ef_co2_ethanol_project=df['distribution_ef_co2_ethanol_project'][i]
    distribution_ef_ch4_ethanol_project=df['distribution_ef_ch4_ethanol_project'][i]
    distribution_ef_n2O_ethanol_project=df['distribution_ef_n2O_ethanol_project'][i]


    distribution_ef_co2_electric_project=df['distribution_ef_co2_electric_project'][i]
    distribution_ef_ch4_electric_project=df['distribution_ef_ch4_electric_project'][i]
    distribution_ef_n2O_electric_project=df['distribution_ef_n2O_electric_project'][i]


    distribution_ef_co2_pellets_project=df['distribution_ef_co2_pellets_project'][i]
    distribution_ef_ch4_pellets_project=df['distribution_ef_ch4_pellets_project'][i]
    distribution_ef_n2O_pellets_project=df['distribution_ef_n2O_pellets_project'][i]

    # Project Processing Emission Factors
    processing_ef_co2_firewood_project=df['processing_ef_co2_firewood_project'][i]
    processing_ef_ch4_firewood_project=df['processing_ef_ch4_firewood_project'][i]
    processing_ef_n2O_firewood_project=df['processing_ef_n2O_firewood_project'][i]


    processing_ef_co2_charcoal_project=df['processing_ef_co2_charcoal_project'][i]
    processing_ef_ch4_charcoal_project=df['processing_ef_ch4_charcoal_project'][i]
    processing_ef_n2O_charcoal_project=df['processing_ef_n2O_charcoal_project'][i]


    processing_ef_co2_kerosene_project=df['processing_ef_co2_kerosene_project'][i]
    processing_ef_ch4_kerosene_project=df['processing_ef_ch4_kerosene_project'][i]
    processing_ef_n2O_kerosene_project=df['processing_ef_n2O_kerosene_project'][i]


    processing_ef_co2_LPG_project=df['processing_ef_co2_LPG_project'][i]
    processing_ef_ch4_LPG_project=df['processing_ef_ch4_LPG_project'][i]
    processing_ef_n2O_LPG_project=df['processing_ef_n2O_LPG_project'][i]


    processing_ef_co2_ethanol_project=df['processing_ef_co2_ethanol_project'][i]
    processing_ef_ch4_ethanol_project=df['processing_ef_ch4_ethanol_project'][i]
    processing_ef_n2O_ethanol_project=df['processing_ef_n2O_ethanol_project'][i]


    processing_ef_co2_electric_project=df['processing_ef_co2_electric_project'][i]
    processing_ef_ch4_electric_project=df['processing_ef_ch4_electric_project'][i]
    processing_ef_n2O_electric_project=df['processing_ef_n2O_electric_project'][i]


    processing_ef_co2_pellets_project=df['processing_ef_co2_pellets_project'][i]
    processing_ef_ch4_pellets_project=df['processing_ef_ch4_pellets_project'][i]
    processing_ef_n2O_pellets_project=df['processing_ef_n2O_pellets_project'][i]
    print(i)

    # Adoption, Usage, and Leakage



    project_adoption=df['project_adoption'][i]


    project_usage=df['project_usage'][i]


    project_adoption_commercial=df['project_adoption_commercial'][i]


    project_usage_commercial=df['project_usage_commercial'][i]

    project_stacking=df['project_stacking'][i]

    project_stacking_commercial=df['project_stacking_commercial'][i]



    leakage_binary=df['leakage_binary'][i]



    leakage_default= df['leakage_default'][i] # put 1 if not being multiplied



    leakage_tCO2e=df['leakage_tCO2e'][i]



    more_than_one_stove_adj=df['more_than_one_stove_adj'][i]

    other_tCO2e_reduction=df['other_tCO2e_reduction'][i]

    #rebound

    rebound=df['rebound'][i]



    # Baseline Emissions
    # Baseline Firewood

    be_firewood_gillwiehl=prc_firewood_baseline*(ncv_firewood_agw*kg_firewood_baseline/1000*((fNRB_project*(ef_combustion_co2_firewood
                                                                                                      +ef_combustion_ch4_firewood*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-fNRB_project)*(ef_combustion_ch4_firewood*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw))))
                                                                                             # renewable emissions use a different ch4 gwp

    # Baseline Charcoal

    be_charcoal_gillwiehl=prc_charcoal_baseline*(ncv_charcoal_agw*kg_charcoal_baseline/1000*((fNRB_project*(ef_combustion_co2_charcoal
                                                                                                      +ef_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-fNRB_project)*(ef_combustion_ch4_charcoal*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw))
                                                                                             # renewable emissions use a different ch4 gwp


                                                                                             +ef_total_not_combustion_co2_charcoal
                                                                                             +ef_total_not_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                             +ef_total_not_combustion_n2O_charcoal*gwp_n2O_agw))
    # Baseline LPG

    be_LPG_gillwiehl=prc_lpg_baseline*(ncv_LPG_agw*kg_lpg_baseline/1000*(ef_total_co2_LPG +ef_total_ch4_LPG*gwp_ch4_agw
                                                                                             +ef_total_n2O_LPG*gwp_n2O_agw))
    # Baseline Kerosene

    be_kerosene_gillwiehl=prc_kerosene_baseline*(ncv_kerosene_agw*kg_kerosene_baseline/1000*(ef_total_co2_kerosene +ef_total_ch4_kerosene*gwp_ch4_agw
                                                                                             +ef_total_n2O_kerosene*gwp_n2O_agw))
    # Baseline Ethanol

    be_ethanol_gillwiehl=prc_ethanol_baseline*(ncv_ethanol_agw*l_ethanol_baseline_commercial*kg_per_liter_ethanol_agw/1000*(ef_total_zeroed_co2_ethanol +ef_total_zeroed_ch4_ethanol*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_ethanol*gwp_n2O_agw))
    # Baseline Pellets

    be_pellets_gillwiehl=prc_pellets_baseline*(ncv_pellets_agw*kg_pellets_baseline/1000*(ef_total_zeroed_co2_pellets +ef_total_zeroed_ch4_pellets*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_pellets*gwp_n2O_agw))
    # Total Baseline
    be_total_domestic_gillwiehl = be_firewood_gillwiehl + be_charcoal_gillwiehl + be_LPG_gillwiehl+ be_ethanol_gillwiehl + be_pellets_gillwiehl

    be_total_domestic_gillwiehl

   # Baseline Commercial Emissions
    # Baseline Commercial Firewood

    be_firewood_commercial_gillwiehl=prc_firewood_baseline_commercial*(ncv_firewood_agw*kg_firewood_baseline_commercial/1000*((fNRB_project*(ef_combustion_co2_firewood
                                                                                                      +ef_combustion_ch4_firewood*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-fNRB_project)*(ef_combustion_ch4_firewood*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw))))
                                                                                             # renewable emissions use a different ch4 gwp

    # Baseline Charcoal

    be_charcoal_commercial_gillwiehl=prc_charcoal_baseline_commercial*(ncv_charcoal_agw*kg_charcoal_baseline_commercial/1000*((fNRB_project*(ef_combustion_co2_charcoal
                                                                                                      +ef_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-fNRB_project)*(ef_combustion_ch4_charcoal*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw))
                                                                                             # renewable emissions use a different ch4 gwp


                                                                                             +ef_total_not_combustion_co2_charcoal
                                                                                             +ef_total_not_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                             +ef_total_not_combustion_n2O_charcoal*gwp_n2O_agw))
    # Baseline LPG

    be_LPG_commercial_gillwiehl=prc_lpg_baseline_commercial*(ncv_LPG_agw*kg_lpg_baseline_commercial/1000*(ef_total_co2_LPG +ef_total_ch4_LPG*gwp_ch4_agw
                                                                                             +ef_total_n2O_LPG*gwp_n2O_agw))
    # Baseline Kerosene

    be_kerosene_commercial_gillwiehl=prc_kerosene_baseline*(ncv_kerosene_agw*kg_kerosene_baseline_commercial/1000*(ef_total_co2_kerosene +ef_total_ch4_kerosene*gwp_ch4_agw
                                                                                             +ef_total_n2O_kerosene*gwp_n2O_agw))
    # Baseline Ethanol

    be_ethanol_commercial_gillwiehl=prc_ethanol_baseline*(ncv_ethanol_agw*l_ethanol_baseline_commercial*kg_per_liter_ethanol_agw/1000*(ef_total_zeroed_co2_ethanol +ef_total_zeroed_ch4_ethanol*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_ethanol*gwp_n2O_agw))
    # Baseline Pellets

    be_pellets_commercial_gillwiehl=prc_pellets_baseline*(ncv_pellets_agw*kg_pellets_baseline_commercial/1000*(ef_total_zeroed_co2_pellets +ef_total_zeroed_ch4_pellets*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_pellets*gwp_n2O_agw))
    # Total Baseline
    be_total_commercial_gillwiehl = be_firewood_commercial_gillwiehl + be_charcoal_commercial_gillwiehl + be_LPG_commercial_gillwiehl+ be_ethanol_commercial_gillwiehl + be_pellets_commercial_gillwiehl

    be_total_commercial_gillwiehl

    # Project Emissions
    # Project Firewood

    pe_firewood_gillwiehl=prc_firewood_project*(ncv_firewood_agw*kg_firewood_project/1000*((fNRB_project*(ef_combustion_co2_firewood
                                                                                                      +ef_combustion_ch4_firewood*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-fNRB_project)*(ef_combustion_ch4_firewood*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw))))
                                                                                             # renewable emissions use a different ch4 gwp

    # Project Charcoal

    pe_charcoal_gillwiehl=prc_charcoal_project*(ncv_charcoal_agw*kg_charcoal_project/1000*((fNRB_project*(ef_combustion_co2_charcoal
                                                                                                      +ef_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-fNRB_project)*(ef_combustion_ch4_charcoal*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw))
                                                                                             # renewable emissions use a different ch4 gwp


                                                                                             +ef_total_not_combustion_co2_charcoal
                                                                                             +ef_total_not_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                             +ef_total_not_combustion_n2O_charcoal*gwp_n2O_agw))
    # Project LPG

    pe_LPG_gillwiehl=prc_lpg_project*(ncv_LPG_agw*kg_lpg_project/1000*(ef_total_co2_LPG +ef_total_ch4_LPG*gwp_ch4_agw
                                                                                             +ef_total_n2O_LPG*gwp_n2O_agw))
    # Project Kerosene

    pe_kerosene_gillwiehl=prc_kerosene_project*(ncv_kerosene_agw*kg_kerosene_project/1000*(ef_total_co2_kerosene +ef_total_ch4_kerosene*gwp_ch4_agw
                                                                                             +ef_total_n2O_kerosene*gwp_n2O_agw))
    # Project Ethanol

    pe_ethanol_gillwiehl=prc_ethanol_project*(ncv_ethanol_agw*l_ethanol_project*kg_per_liter_ethanol_agw/1000*(ef_total_zeroed_co2_ethanol +ef_total_zeroed_ch4_ethanol*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_ethanol*gwp_n2O_agw))
    # Project Pellets

    pe_pellets_gillwiehl=prc_pellets_project*(ncv_pellets_agw*kg_pellets_project/1000*(ef_total_zeroed_co2_pellets +ef_total_zeroed_ch4_pellets*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_pellets*gwp_n2O_agw))
    # Total project
    pe_total_domestic_gillwiehl = pe_firewood_gillwiehl + pe_charcoal_gillwiehl + pe_LPG_gillwiehl+ pe_ethanol_gillwiehl + pe_pellets_gillwiehl

    pe_total_domestic_gillwiehl

    # Project Commercial Emissions
    # Project Commercial Firewood

    pe_firewood_commercial_gillwiehl=prc_firewood_project_commercial*(ncv_firewood_agw*kg_firewood_project_commercial/1000*((fNRB_project*(ef_combustion_co2_firewood
                                                                                                      +ef_combustion_ch4_firewood*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-fNRB_project)*(ef_combustion_ch4_firewood*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_firewood*gwp_n2O_agw))))
                                                                                             # renewable emissions use a different ch4 gwp

    # Project Charcoal

    pe_charcoal_commercial_gillwiehl=prc_charcoal_project_commercial*(ncv_charcoal_agw*kg_charcoal_project_commercial/1000*((fNRB_project*(ef_combustion_co2_charcoal
                                                                                                      +ef_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw)
                                                                                              # non-renewable
                                                                                              +(1-fNRB_project)*(ef_combustion_ch4_charcoal*gwp_ch4_renewable
                                                                                                      + ef_combustion_n2O_charcoal*gwp_n2O_agw))
                                                                                             # renewable emissions use a different ch4 gwp


                                                                                             +ef_total_not_combustion_co2_charcoal
                                                                                             +ef_total_not_combustion_ch4_charcoal*gwp_ch4_agw
                                                                                             +ef_total_not_combustion_n2O_charcoal*gwp_n2O_agw))
    # Project LPG

    pe_LPG_commercial_gillwiehl=prc_lpg_project_commercial*(ncv_LPG_agw*kg_lpg_project_commercial/1000*(ef_total_co2_LPG +ef_total_ch4_LPG*gwp_ch4_agw
                                                                                             +ef_total_n2O_LPG*gwp_n2O_agw))
    # Project Kerosene

    pe_kerosene_commercial_gillwiehl=prc_kerosene_project_commercial*(ncv_kerosene_agw*kg_kerosene_project_commercial/1000*(ef_total_co2_kerosene +ef_total_ch4_kerosene*gwp_ch4_agw
                                                                                             +ef_total_n2O_kerosene*gwp_n2O_agw))
    # Project Ethanol

    pe_ethanol_commercial_gillwiehl=prc_ethanol_project_commercial*(ncv_ethanol_agw*l_ethanol_project_commercial*kg_per_liter_ethanol_agw/1000*(ef_total_zeroed_co2_ethanol +ef_total_zeroed_ch4_ethanol*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_ethanol*gwp_n2O_agw))
    # Project Ethanol

    pe_pellets_commercial_gillwiehl=prc_pellets_project_commercial*(ncv_pellets_agw*kg_pellets_project_commercial/1000*(ef_total_zeroed_co2_pellets +ef_total_zeroed_ch4_pellets*gwp_ch4_agw
                                                                                             +ef_total_zeroed_n2O_pellets*gwp_n2O_agw))
    # Total project
    pe_total_commercial_gillwiehl = pe_firewood_commercial_gillwiehl + pe_charcoal_commercial_gillwiehl + pe_LPG_commercial_gillwiehl+ pe_ethanol_commercial_gillwiehl + pe_pellets_commercial_gillwiehl

    pe_total_commercial_gillwiehl




    # Totalling Baseline and Project Emissions

    be_total_agw=be_total_domestic_gillwiehl*project_adoption*project_usage*(1-project_stacking)

    be_total_agw=be_total_agw+be_total_commercial_gillwiehl*project_adoption_commercial*project_usage_commercial*(1-project_stacking_commercial)

    pe_total_agw=pe_total_domestic_gillwiehl*project_adoption*project_usage*(1-project_stacking)
    pe_total_agw=pe_total_agw+pe_total_commercial_gillwiehl*project_adoption_commercial*project_usage*(1-project_stacking_commercial)

    PEs_agw=(pe_total_agw)*more_than_one_stove_adj # Project Emission Reductions (PERs) in (tCO2e)
    BEs_agw=(be_total_agw)*more_than_one_stove_adj # Project Emission Reductions (PERs) in (tCO2e)

    VERs_agw=(be_total_agw-pe_total_agw)*more_than_one_stove_adj*leakage_default-(leakage_tCO2e+other_tCO2e_reduction)/(n_devices_domestic+n_devices_commercial)/(project_days+project_days_commercial) # Project Emission Reductions (PERs) in (tCO2e) # Project Emission Reductions (PERs) in (tCO2e)




    # Protocol Analysis:

    all_VERS_agw.append(VERs_agw)
    ALL_N=ALL_N+1

    #GS Metered
    if df.protocol_type[i]=='GS Metered':
        GS_Meter_N=GS_Meter_N+1
        GS_Meter_VERs_agw.append(VERs_agw)

    #GS
    if df.protocol_type[i]=='GS TPDDTEC':
        GS_N=GS_N+1
        GS_VERs_agw.append(VERs_agw)


    # CDM II G

    if df.protocol_type[i]=='CDM AMS II G':
        CDM_II_G_N=CDM_II_G_N+1
        CDM_II_G_VERs_agw.append(VERs_agw)

    # CDM I E
    if df.protocol_type[i]=='CDM AMS I E':
        CDM_I_E_N=CDM_I_E_N+1
        CDM_I_E_VERs_agw.append(VERs_agw)

    #bangladesh
    if df.country[i]=='Bangladesh':
        bangladesh_N=bangladesh_N+1
        bangladesh_VERs_agw.append(VERs_agw)

     #burkina faso
    if df.country[i]=='Burkina Faso':
        burkinafaso_N=burkinafaso_N+1
        burkinafaso_VERs_agw.append(VERs_agw)

    #china
    if df.country[i]=='China':
        china_N=china_N+1
        china_VERs_agw.append(VERs_agw)

    #ethiopia
    if df.country[i]=='Ethiopia':
        ethiopia_N=ethiopia_N+1
        ethiopia_VERs_agw.append(VERs_agw)

    #ghana
    if df.country[i]=='Ghana':
        ghana_N=ghana_N+1
        ghana_VERs_agw.append(VERs_agw)

    #guatemala
    if df.country[i]=='Guatemala':
        guatemala_N=guatemala_N+1
        guatemala_VERs_agw.append(VERs_agw)

    #haiti
    if df.country[i]=='Haiti':
        haiti_N=haiti_N+1
        haiti_VERs_agw.append(VERs_agw)

    #honduras
    if df.country[i]=='Honduras':
        honduras_N=honduras_N+1
        honduras_VERs_agw.append(VERs_agw)

    #india
    if df.country[i]=='India':
        india_N=india_N+1
        india_VERs_agw.append(VERs_agw)

    #kenya
    if df.country[i]=='Kenya':
        kenya_N=kenya_N+1
        kenya_VERs_agw.append(VERs_agw)

    #lesotho
    if df.country[i]=='Lesotho':
        lesotho_N=lesotho_N+1
        lesotho_VERs_agw.append(VERs_agw)

    #malawi
    if df.country[i]=='Malawi':
        malawi_N=malawi_N+1
        malawi_VERs_agw.append(VERs_agw)

    #mali
    if df.country[i]=='Mali':
        mali_N=mali_N+1
        mali_VERs_agw.append(VERs_agw)

    #mexico
    if df.country[i]=='Mexico':
        mexico_N=mexico_N+1
        mexico_VERs_agw.append(VERs_agw)

    #mozambique
    if df.country[i]=='Mozambique':
        mozambique_N=mozambique_N+1
        mozambique_VERs_agw.append(VERs_agw)

    #myanmar
    if df.country[i]=='Myanmar':
        myanmar_N=myanmar_N+1
        myanmar_VERs_agw.append(VERs_agw)

    #nepal
    if df.country[i]=='Nepal':
        nepal_N=nepal_N+1
        nepal_VERs_agw.append(VERs_agw)

    #nigeria
    if df.country[i]=='Nigeria':
        nigeria_N=nigeria_N+1
        nigeria_VERs_agw.append(VERs_agw)

    #rwanda
    if df.country[i]=='Rwanda':
        rwanda_N=rwanda_N+1
        rwanda_VERs_agw.append(VERs_agw)

    #sudan
    if df.country[i]=='Sudan':
        sudan_N=sudan_N+1
        sudan_VERs_agw.append(VERs_agw)

    #uganda
    if df.country[i]=='Uganda':
        uganda_N=uganda_N+1
        uganda_VERs_agw.append(VERs_agw)

    #zambia
    if df.country[i]=='Zambia':
        zambia_N=zambia_N+1
        zambia_VERs_agw.append(VERs_agw)

    #firewood
    if df.type[i]=='firewood':
        firewood_N=firewood_N+1
        firewood_VERs_agw.append(VERs_agw)

    #charcoal
    if df.type[i]=='charcoal':
        charcoal_N=charcoal_N+1
        charcoal_VERs_agw.append(VERs_agw)

    #biomass
    if df.type[i]=='biomass':
        biomass_N=charcoal_N+1
        biomass_VERs_agw.append(VERs_agw)

    #switch
    if df.type[i]=='switch':
        switch_N=switch_N+1
        switch_VERs_agw.append(VERs_agw)

    #gs_lpg
    if df.stove[i]=='gs_lpg':
        gs_lpg_N=gs_lpg_N+1
        gs_lpg_VERs_agw.append(VERs_agw)

    #gs_imp_firewood
    if df.stove[i]=='gs_imp_firewood':
        gs_imp_firewood_N=gs_imp_firewood_N+1
        gs_imp_firewood_VERs_agw.append(VERs_agw)

    #gs_imp_charcoal
    if df.stove[i]=='gs_imp_charcoal':
        gs_imp_charcoal_N=gs_imp_charcoal_N+1
        gs_imp_charcoal_VERs_agw.append(VERs_agw)

    #cdm_imp_firewood
    if df.stove[i]=='cdm_imp_firewood':
        cdm_imp_firewood_N=cdm_imp_firewood_N+1
        cdm_imp_firewood_VERs_agw.append(VERs_agw)

    #cdm_imp_charcoal
    if df.stove[i]=='cdm_imp_charcoal':
        cdm_imp_charcoal_N=cdm_imp_charcoal_N+1
        cdm_imp_charcoal_VERs_agw.append(VERs_agw)

    #gs_simple_firewood
    if df.stove[i]=='gs_simple_firewood':
        gs_simple_firewood_N=gs_simple_firewood_N+1
        gs_simple_firewood_VERs_agw.append(VERs_agw)

    #cameroon
    if df.country[i]=='Cameroon':
        cameroon_N=cameroon_N+1
        cameroon_VERs_agw.append(VERs_agw)

    #peru
    if df.country[i]=='Peru':
        peru_N=peru_N+1
        peru_VERs_agw.append(VERs_agw)

    print(protocol_id)

    #Times Over Credited
    ratio_analysis_project=VERs_agw/VERs
    ratio_analysis_project_mean=np.mean(VERs_agw)/VERs*100
    ratio_analysis_project_mean
    ratio_analysis_project_mean_compilation.append(ratio_analysis_project_mean)


    ratio_project_analysis=VERs/VERs_agw
    ratio_project_analysis_mean=VERs/np.mean(VERs_agw)
    ratio_project_analysis_mean
    ratio_project_analysis_mean_compilation.append(ratio_project_analysis_mean)
    z=1.96

    sd_VERs_agw=np.std(VERs_agw)
    sd_ratio_analysis_project=np.std(ratio_analysis_project)
    sd_ratio_project_analysis=np.std(ratio_project_analysis)


    ratio_analysis_project_lower=ratio_analysis_project_mean - z*sd_ratio_analysis_project/np.sqrt(N)
    ratio_analysis_project_upper=ratio_analysis_project_mean + z*sd_ratio_analysis_project/np.sqrt(N)

    ratio_analysis_project_lower_compilation.append(ratio_analysis_project_lower)
    ratio_analysis_project_upper_compilation.append(ratio_analysis_project_upper)


    ratio_project_analysis_lower=ratio_project_analysis_mean - z*sd_ratio_project_analysis/np.sqrt(N)
    ratio_project_analysis_upper=ratio_project_analysis_mean + z*sd_ratio_project_analysis/np.sqrt(N)

    ratio_project_analysis_lower_compilation.append(ratio_project_analysis_lower)
    ratio_project_analysis_upper_compilation.append(ratio_project_analysis_upper)

    print('We find that this project reports emissions which are',round(ratio_project_analysis_mean,2),'[',round(ratio_project_analysis_lower,2),' , ',round(ratio_project_analysis_upper,2),']','times overcredited.')
    print('Our analysis finds emissions which are',round(ratio_analysis_project_mean,2),'[',round(ratio_analysis_project_lower,2),' , ',round(ratio_analysis_project_upper,2),']','% of the verified credits in the PDD.')

    i=i+1

1
PoA 10415 CPA MM 03
We find that this project reports emissions which are 0.67 [ 0.67  ,  0.67 ] times overcredited.
Our analysis finds emissions which are 149.87 [ 149.87  ,  149.87 ] % of the verified credits in the PDD.
2
PoA 10415 CPA MM 03
We find that this project reports emissions which are 0.67 [ 0.67  ,  0.67 ] times overcredited.
Our analysis finds emissions which are 149.87 [ 149.87  ,  149.87 ] % of the verified credits in the PDD.
3
PoA 10415 CPA2
We find that this project reports emissions which are 0.67 [ 0.67  ,  0.67 ] times overcredited.
Our analysis finds emissions which are 149.87 [ 149.87  ,  149.87 ] % of the verified credits in the PDD.


In [8]:
df_all_VERS_agw=pd.DataFrame(all_VERS_agw)
df_all_VERS_agw.to_excel('df_all_VERs_agw_both_EF_charcoal.xlsx')

# Total over-crediting

In [9]:
ratio_project_analysis_mean_compilation=pd.DataFrame(ratio_project_analysis_mean_compilation)
ratio_project_analysis_lower_compilation=pd.DataFrame(ratio_project_analysis_lower_compilation)
ratio_project_analysis_upper_compilation=pd.DataFrame(ratio_project_analysis_upper_compilation)
ratio_analysis_project_mean_compilation=pd.DataFrame(ratio_analysis_project_mean_compilation)
ratio_analysis_project_lower_compilation=pd.DataFrame(ratio_analysis_project_lower_compilation)
ratio_analysis_project_upper_compilation=pd.DataFrame(ratio_analysis_project_upper_compilation)

In [10]:
PoAs=pd.DataFrame(df['PoA'])
ids=pd.DataFrame(df['protocol_id'])
category=pd.DataFrame(df['stove'])
country=pd.DataFrame(df['country'])
total_credits=pd.DataFrame(df['total_verified_credits'])
stove_days=pd.DataFrame(df['stove_days'])

df_info=PoAs.join(ids)
df_info=df_info.join(category)
df_info=df_info.join(country)
df_info=df_info.join(total_credits)
df_info=df_info.join(stove_days)
df_info=pd.DataFrame(df_info)
df_info=pd.concat([df_info[1:].reset_index(drop=True),ratio_project_analysis_lower_compilation], axis=1)
df_info= pd.concat([df_info.reset_index(drop=True),ratio_project_analysis_mean_compilation], axis=1)
df_info= pd.concat([df_info.reset_index(drop=True),ratio_project_analysis_upper_compilation], axis=1)
df_info.columns = ['PoA','protocol_id', 'stove', 'country','total_verified_credits','stove_days','lower','mean','upper']
df_info['stove_days_mean_overcrediting']=df_info['mean']*df_info['stove_days']
df_info
df_project_weighted_stove_days_numerator=df_info.groupby('PoA')['stove_days_mean_overcrediting'].sum()
df_project_weighted_stove_days_numerator=pd.DataFrame(df_project_weighted_stove_days_numerator)


df_project_weighted_stove_days_denominator=df_info.groupby('PoA')['stove_days'].sum()
df_project_weighted_stove_days_denominator=pd.DataFrame(df_project_weighted_stove_days_denominator)
df_project_weighted_stove_days=df_project_weighted_stove_days_numerator['stove_days_mean_overcrediting']/df_project_weighted_stove_days_denominator['stove_days']
df_project_weighted_stove_days=pd.DataFrame(df_project_weighted_stove_days)
df_project_weighted_stove_days.mean() #This is the average project over-crediting
print(df_project_weighted_stove_days.mean())
df_project_weighted_stove_days
protocol_ids=pd.DataFrame(df_info['PoA']) #.unique())
protocol_ids.columns=['PoA']

df_total=df_project_weighted_stove_days.merge(df_info,left_on='PoA', right_on='PoA',how='left')
df_total.columns=['protocol_id','project_over_crediting','PoA','category','country','total_verified_credits','stove_days','lower','mean','upper','stove_days_mean_overcrediting']

df_total['over_credited_total']=df_total['total_verified_credits']/df_total['project_over_crediting']
total_verified_credits=df_total['total_verified_credits'].sum()
total_over_crediting =df_total['over_credited_total'].sum()
reported_value=total_verified_credits/total_over_crediting
print(total_verified_credits/total_over_crediting)
sd=df_total['total_verified_credits'].std()/df_total['over_credited_total'].std()
ci=sd*1.96/np.sqrt(len(protocol_ids))
ci
ci_lower=reported_value-ci
ci_upper=reported_value+ci
print(reported_value,'[',ci_lower,',',ci_upper,']')

0    0.667262
dtype: float64
0.6672616421576372
0.6672616421576372 [ -0.08781598773282495 , 1.4223392720480992 ]


In [11]:
PoA=pd.DataFrame(df_info['PoA'].unique())
PoA.columns=['PoA']

df_total=df_project_weighted_stove_days.merge(df_info,left_on='PoA', right_on='PoA',how='left')
df_total.columns=['PoA', 'project_over_crediting','protocol_id','category','country','total_verified_credits','stove_days','lower','mean','upper','stove_days_mean_overcrediting']
df_total['over_credited_total']=df_total['total_verified_credits']/df_total['project_over_crediting']
total_verified_credits=df_total['total_verified_credits'].sum()
total_over_crediting =df_total['over_credited_total'].sum()
reported_value=total_verified_credits/total_over_crediting
print(total_verified_credits/total_over_crediting)
sd=df_total['total_verified_credits'].std()/df_total['over_credited_total'].std()
ci=sd*1.96/np.sqrt(len(protocol_ids))
ci
ci_lower=reported_value-ci
ci_upper=reported_value+ci
print(reported_value,'[',ci_lower,',',ci_upper,']')

0.6672616421576372
0.6672616421576372 [ -0.08781598773282495 , 1.4223392720480992 ]


# PoA, country and Methodology-stove combination analysis

In [12]:
# PoA weighted average
df_project_weighted_stove_days_numerator=df_info.groupby('PoA')['stove_days_mean_overcrediting'].sum()
df_project_weighted_stove_days_numerator=pd.DataFrame(df_project_weighted_stove_days_numerator)


df_project_weighted_stove_days_denominator=df_info.groupby('PoA')['stove_days'].sum()
df_project_weighted_stove_days_denominator=pd.DataFrame(df_project_weighted_stove_days_denominator)
df_project_weighted_stove_days=df_project_weighted_stove_days_numerator['stove_days_mean_overcrediting']/df_project_weighted_stove_days_denominator['stove_days']
df_project_weighted_stove_days=pd.DataFrame(df_project_weighted_stove_days)
df_project_weighted_stove_days.mean() #This is the average project over-crediting
print(df_project_weighted_stove_days.mean())
df_project_weighted_stove_days
df_project_weighted_stove_days.to_excel("df_project_overcrediting.xlsx")

0    0.667262
dtype: float64


In [13]:
# PoA
my_tuple=df_total.groupby('PoA')['total_verified_credits'].unique()
countries=pd.DataFrame(my_tuple.index)

df_PoA_totals = pd.DataFrame(list(my_tuple))
df_PoA_totals.index=my_tuple.index

df_PoA_totals['total']=df_PoA_totals.sum(axis=1)
df_PoA_totals
my_tuple2=df_total.groupby('PoA')['over_credited_total'].unique()
countries=pd.DataFrame(my_tuple2.index)

df_PoA_totals_analysis = pd.DataFrame(list(my_tuple2))
df_PoA_totals_analysis.index=my_tuple2.index

df_PoA_totals_analysis['total']=df_PoA_totals_analysis.sum(axis=1)
df_PoA_totals_analysis
df_PoA_final=df_PoA_totals/df_PoA_totals_analysis
df_PoA_final.shape[1]-1
df_PoA_final['sd']=df_PoA_final.std(axis=1)
df_PoA_final['count']=df_PoA_final.iloc[:,0:df_PoA_final.shape[1]-1].count(axis=1)
df_PoA_final['lower']=df_PoA_final['total']-df_PoA_final['sd']*1.96/np.sqrt(df_PoA_final['count'])
df_PoA_final['upper']=df_PoA_final['total']+df_PoA_final['sd']*1.96/np.sqrt(df_PoA_final['count'])
results_export=df_PoA_final.iloc[:,(df_PoA_final.shape[1]-5):(df_PoA_final.shape[1])]
results_export.to_excel('all_results_categories.xlsx')

df_PoA_final=results_export
df_PoA_final

,total,sd,count,lower,upper
PoA,,,,,
10415 TRANSITION,0.667262,6.409876e-17,4,0.667262,0.667262
